In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
sia = SentimentIntensityAnalyzer()
plt.style.use('ggplot')
from scipy.special import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch
from textblob import TextBlob
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
# Read dataset
df = pd.read_csv('Reviews.csv')

# Keep 200 reviews for quicker results
df = df.head(200)


In [11]:
df.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [12]:
# Delete unwanted columns
df.drop(['UserId', 'ProfileName', 'Time', 'HelpfulnessNumerator', 'HelpfulnessDenominator'], axis=1, inplace=True)

In [13]:
df.head()

,Id,ProductId,Score,Summary,Text
0,1,B001E4KFG0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,2,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,5,Great taffy,Great taffy at a great price. There was a wid...


In [14]:
# Initiate new columns for Polarity (in words) and Sentiment Score in 0, 1, -1 for Neutral, Positive, Negative respectively. Scores to be given according to dataset given scores
df['Polarity'] = 0
df['SentimentScore'] = 0

In [15]:
# In the new columns, give Neutral for Scores that are 3 in the dataset, give Negative for 1, 2 and Positive for 4, 5 in order to cluster all 5 scores into three.
for i in range(len(df)):
  if df.Score[i] == 3:
    df.Polarity[i] = 'Neutral'
    df.SentimentScore[i] = 0
  elif df.Score[i] == 4 or df.Score[i] == 5:
    df.Polarity[i] = 'Positive'
    df.SentimentScore[i] = 1
  elif df.Score[i] == 1 or df.Score[i] == 2:
    df.Polarity[i] = 'Negative'
    df.SentimentScore[i] = -1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipyk

In [16]:
df.head()

,Id,ProductId,Score,Summary,Text,Polarity,SentimentScore
0,1,B001E4KFG0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Positive,1
1,2,B00813GRG4,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Negative,-1
2,3,B000LQOCH0,4,"""Delight"" says it all",This is a confection that has been around a fe...,Positive,1
3,4,B000UA0QIQ,2,Cough Medicine,If you are looking for the secret ingredient i...,Negative,-1
4,5,B006K2ZZ7K,5,Great taffy,Great taffy at a great price. There was a wid...,Positive,1


In [17]:
# VADER Sentiment Analysis

In [18]:
# Check results given by Vader
sia.polarity_scores('I am delighted with the idea, it is amazing')

{'neg': 0.0, 'neu': 0.458, 'pos': 0.542, 'compound': 0.7964}

In [19]:
# Initiate Columns to store results from Vader Analysis
df['VaderPos'] = 0.0
df['VaderNeg'] = 0.0
df['VaderNeu'] = 0.0
df['VaderComp'] = 0.0
df['PolarityVader'] = 0
df['ScoreVader'] = 0

In [20]:
# Vader Analysis. Using try, except because certain lines are too long for the transformers to be used on them
for i in range(len(df)):
  try:
    text = df.Text[i] # Get each row as text
    # Store Positive, Negative, Neutral scores and compound score for each row
    df['VaderPos'][i] = sia.polarity_scores(text)['pos']
    df['VaderNeg'][i] = sia.polarity_scores(text)['neg']
    df['VaderNeu'][i] = sia.polarity_scores(text)['neu']
    df['VaderComp'][i] = sia.polarity_scores(text)['compound']
    # Classify each comment as negative, positive and neutral according to scores. 0.33 was decided as a benchmark so that it gives equal spaces to all 3 polarities. 
    if sia.polarity_scores(text)['compound'] < -0.33:
      df['PolarityVader'][i] = 'Negative'
      df['ScoreVader'][i] = -1
    elif sia.polarity_scores(text)['compound'] > 0.33:
      df['PolarityVader'][i] = 'Positive'
      df['ScoreVader'][i] = 1
    else:
      df['PolarityVader'][i] = 'Neutral'
      df['ScoreVader'][i] = 0
  except:
    print(f'Broke for id {i + 1}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value

In [21]:
df.head()

,Id,ProductId,Score,Summary,Text,Polarity,SentimentScore,VaderPos,VaderNeg,VaderNeu,VaderComp,PolarityVader,ScoreVader
0,1,B001E4KFG0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Positive,1,0.305,0.000,0.695,0.9441,Positive,1
1,2,B00813GRG4,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Negative,-1,0.000,0.138,0.862,-0.5664,Negative,-1
2,3,B000LQOCH0,4,"""Delight"" says it all",This is a confection that has been around a fe...,Positive,1,0.155,0.091,0.754,0.8265,Positive,1
3,4,B000UA0QIQ,2,Cough Medicine,If you are looking for the secret ingredient i...,Negative,-1,0.000,0.000,1.000,0.0000,Neutral,0
4,5,B006K2ZZ7K,5,Great taffy,Great taffy at a great price. There was a wid...,Positive,1,0.448,0.000,0.552,0.9468,Positive,1


In [22]:
print(f'Vader Sentiment Analysis gives:\nAccuracy Score: {accuracy_score(df.SentimentScore, df.ScoreVader)}')

Vader Sentiment Analysis gives:
Accuracy Score: 0.78


In [23]:
print(confusion_matrix(df.SentimentScore, df.ScoreVader))

[[  6   9   4]
 [  3   3  12]
 [  8   8 147]]


In [24]:
print(classification_report(df.SentimentScore, df.ScoreVader))

              precision    recall  f1-score   support

          -1       0.35      0.32      0.33        19
           0       0.15      0.17      0.16        18
           1       0.90      0.90      0.90       163

    accuracy                           0.78       200
   macro avg       0.47      0.46      0.46       200
weighted avg       0.78      0.78      0.78       200



In [25]:
# BERT Analysis

In [26]:
# Initiate columns for BERT results to be stored in.
df['PolarityBert'] = 0
df['SentimentBert'] = 0
df['SentimentBertScore'] = 0

In [27]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
for i in range(len(df)):
  try:
    text = df.Text[i]
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    # BERT results are scores from 0 to 4. Adding +1 to match results of the dataset and make them 1 to 5.
    df.PolarityBert[i] = int(torch.argmax(result.logits))+1
  except:
    print(f'Broke for id {i + 1}')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
Token indices sequence length is longer than the specified maximum sequence length for this model (611 > 512). Running this sequence through the model will result in indexing errors


Broke for id 83
Broke for id 187


In [28]:
df.head(5)

,Id,ProductId,Score,Summary,Text,Polarity,SentimentScore,VaderPos,VaderNeg,VaderNeu,VaderComp,PolarityVader,ScoreVader,PolarityBert,SentimentBert,SentimentBertScore
0,1,B001E4KFG0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Positive,1,0.305,0.000,0.695,0.9441,Positive,1,5,0,0
1,2,B00813GRG4,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Negative,-1,0.000,0.138,0.862,-0.5664,Negative,-1,1,0,0
2,3,B000LQOCH0,4,"""Delight"" says it all",This is a confection that has been around a fe...,Positive,1,0.155,0.091,0.754,0.8265,Positive,1,5,0,0
3,4,B000UA0QIQ,2,Cough Medicine,If you are looking for the secret ingredient i...,Negative,-1,0.000,0.000,1.000,0.0000,Neutral,0,5,0,0
4,5,B006K2ZZ7K,5,Great taffy,Great taffy at a great price. There was a wid...,Positive,1,0.448,0.000,0.552,0.9468,Positive,1,5,0,0


In [29]:
# As done for the dataset's given results, the 5 results are classified into 3 in order to check the results.
for i in range(len(df)):
  if df.PolarityBert[i] == 3:
    df.SentimentBert[i] = 'Neutral'
    df.SentimentBertScore[i] = 0
  elif df.PolarityBert[i] == 4 or df.PolarityBert[i] == 5:
    df.SentimentBert[i] = 'Positive'
    df.SentimentBertScore[i] = 1
  elif df.PolarityBert[i] == 1 or df.PolarityBert[i] == 2:
    df.SentimentBert[i] = 'Negative'
    df.SentimentBertScore[i] = -1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipyk

In [30]:
print(f'BERT Sentiment Analysis gives:\nAccuracy Score of grained Sentiment Analysis: {accuracy_score(df.Score, df.PolarityBert)}\nAccuracy Score of binary Sentiment Analysis: {accuracy_score(df.SentimentScore, df.SentimentBertScore)}')

BERT Sentiment Analysis gives:
Accuracy Score of grained Sentiment Analysis: 0.64
Accuracy Score of binary Sentiment Analysis: 0.82


In [31]:
print(confusion_matrix(df.SentimentScore, df.SentimentBertScore))

[[ 14   3   2]
 [  7   9   2]
 [ 11  11 141]]


In [32]:
# Results for the binary classification
print(classification_report(df.SentimentScore, df.SentimentBertScore))

              precision    recall  f1-score   support

          -1       0.44      0.74      0.55        19
           0       0.39      0.50      0.44        18
           1       0.97      0.87      0.92       163

    accuracy                           0.82       200
   macro avg       0.60      0.70      0.63       200
weighted avg       0.87      0.82      0.84       200



In [33]:
# Results for the fine grained sentiment analysis.
print(classification_report(df.Score, df.PolarityBert))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.67      0.83      0.74        12
           2       0.12      0.29      0.17         7
           3       0.43      0.50      0.46        18
           4       0.29      0.46      0.36        26
           5       0.91      0.69      0.79       137

    accuracy                           0.64       200
   macro avg       0.40      0.46      0.42       200
weighted avg       0.75      0.64      0.68       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
# TEXT BLOB Analysis

In [35]:
# Initializing Columns for Text Blob results to be stored in.
df['PolarityBlob'] = 0.0
df['SubjectivityBlob'] = 0.0
df['SentimentBlob'] = 0
df['SentimentBlobScore'] = 0

In [36]:
# TextBlob analysis for each row.
for i in range(len(df)):
  try:
    text = df.Text[i]
    blob = TextBlob(text)
    df['PolarityBlob'][i] = blob.sentiment[0]
    df['SubjectivityBlob'][i] = blob.sentiment[1]
  except:
    print(f'Broke for id {i + 1}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [37]:
# Classify each row's text as neutral, positive or negative according to results taken from TextBlob
for i in range(len(df)):
  if df.PolarityBlob[i] == 0.0:
    df.SentimentBlob[i] = 'Neutral'
    df.SentimentBlobScore[i] = 0
  elif df.PolarityBlob[i] > 0:
    df.SentimentBlob[i] = 'Positive'
    df.SentimentBlobScore[i] = 1
  elif df.PolarityBlob[i] < 0:
    df.SentimentBlob[i] = 'Negative'
    df.SentimentBlobScore[i] = -1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipyk

In [38]:
df.head()

,Id,ProductId,Score,Summary,Text,Polarity,SentimentScore,VaderPos,VaderNeg,VaderNeu,VaderComp,PolarityVader,ScoreVader,PolarityBert,SentimentBert,SentimentBertScore,PolarityBlob,SubjectivityBlob,SentimentBlob,SentimentBlobScore
0,1,B001E4KFG0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Positive,1,0.305,0.000,0.695,0.9441,Positive,1,5,Positive,1,0.450000,0.433333,Positive,1
1,2,B00813GRG4,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Negative,-1,0.000,0.138,0.862,-0.5664,Negative,-1,1,Negative,-1,-0.033333,0.762963,Negative,-1
2,3,B000LQOCH0,4,"""Delight"" says it all",This is a confection that has been around a fe...,Positive,1,0.155,0.091,0.754,0.8265,Positive,1,5,Positive,1,0.133571,0.448571,Positive,1
3,4,B000UA0QIQ,2,Cough Medicine,If you are looking for the secret ingredient i...,Negative,-1,0.000,0.000,1.000,0.0000,Neutral,0,5,Positive,1,0.166667,0.533333,Positive,1
4,5,B006K2ZZ7K,5,Great taffy,Great taffy at a great price. There was a wid...,Positive,1,0.448,0.000,0.552,0.9468,Positive,1,5,Positive,1,0.483333,0.637500,Positive,1


In [39]:
print(f'TextBlob Sentiment Analysis gives:\nAccuracy Score: {accuracy_score(df.SentimentScore, df.SentimentBlobScore)}')

TextBlob Sentiment Analysis gives:
Accuracy Score: 0.79


In [40]:
print(confusion_matrix(df.SentimentScore, df.SentimentBlobScore))

[[  7   2  10]
 [  2   0  16]
 [ 10   2 151]]


In [41]:
print(classification_report(df.SentimentScore, df.SentimentBlobScore))

              precision    recall  f1-score   support

          -1       0.37      0.37      0.37        19
           0       0.00      0.00      0.00        18
           1       0.85      0.93      0.89       163

    accuracy                           0.79       200
   macro avg       0.41      0.43      0.42       200
weighted avg       0.73      0.79      0.76       200



In [42]:
# Simple Pipeline Analysis

In [43]:
# Initialize Columns for Pipeline results to be stored in.
df['PolarityPipeline'] = 0
df['ScorePipeline'] = 0.0
df['SentimentPipelineScore'] = 0.0

In [44]:
# Apply the pipeline model for each row.
sent_pipeline = pipeline('sentiment-analysis')
for i in range(len(df)):
  try:
    text = df.Text[i]
    df['PolarityPipeline'][i] = sent_pipeline(text)[0]['label']
    df['ScorePipeline'][i] = sent_pipeline(text)[0]['score']
  except:
    print(f'Broke for id {i + 1}')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
Token indices sequence length is longer tha

Broke for id 83
Broke for id 187


In [45]:
df['PolarityPipeline'].unique()

array(['POSITIVE', 'NEGATIVE', 0], dtype=object)

In [46]:
# Classify per pipeline's results.
for i in range(len(df)):
  if df.PolarityPipeline[i] == 0:
    df.SentimentPipelineScore[i] = 0
    df.PolarityPipeline[i] = 'Neutral'
  elif df.PolarityPipeline[i] == 'POSITIVE':
    df.SentimentPipelineScore[i] = 1
    df.PolarityPipeline[i] = 'Positive'
  elif df.PolarityPipeline[i] == 'NEGATIVE':
    df.SentimentPipelineScore[i] = -1
    df.PolarityPipeline[i] = 'Negative'
  else:
    print(i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [47]:
df.SentimentPipelineScore.unique()

array([ 1., -1.,  0.])

In [48]:
# Roberta Analysis

In [49]:
# Initialize Columns for the Roberta Model's results to be stored in.
df['RobertaPos'] = 0.0
df['RobertaNeg'] = 0.0
df['RobertaNeu'] = 0.0
df['PolarityRoberta'] = 0
df['ScoreRoberta'] = 0

In [51]:
# Apply Roberta in every row.
tokenizer = AutoTokenizer.from_pretrained(f"cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained(f"cardiffnlp/twitter-roberta-base-sentiment")
for i in range(len(df)):
  try:
    text = df.Text[i]
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text) # Tensored output
    scores = output[0][0].detach().numpy() # Transform the output from Tensored to numpy array
    scores = softmax(scores)
    df['RobertaPos'][i] = scores[2]
    df['RobertaNeg'][i] = scores[0]
    df['RobertaNeu'][i] = scores[1]
  except:
    print(f'Broke for id {i + 1}')

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Broke for id 83
Broke for id 187


In [52]:
# Classify each row according to which polarity has higher percentage given by Roberta.
for i in range(len(df)):
  if df.RobertaNeu[i] >= df.RobertaPos[i] and df.RobertaNeu[i] >= df.RobertaNeg[i]:
    df.PolarityRoberta[i] = 'Neutral'
    df.ScoreRoberta[i] = 0
  elif df.RobertaPos[i] >= df.RobertaNeg[i] and df.RobertaPos[i] >= df.RobertaNeu[i]:
    df.PolarityRoberta[i] = 'Positive'
    df.ScoreRoberta[i] = 1
  elif df.RobertaNeg[i] >= df.RobertaPos[i] and df.RobertaNeg[i] >= df.RobertaNeu[i]:
    df.PolarityRoberta[i] = 'Negative'
    df.ScoreRoberta[i] = -1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/i

In [53]:
df.head()

,Id,ProductId,Score,Summary,Text,Polarity,SentimentScore,VaderPos,VaderNeg,VaderNeu,...,SentimentBlob,SentimentBlobScore,PolarityPipeline,ScorePipeline,SentimentPipelineScore,RobertaPos,RobertaNeg,RobertaNeu,PolarityRoberta,ScoreRoberta
0,1,B001E4KFG0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Positive,1,0.305,0.000,0.695,...,Positive,1,Positive,0.998385,1.0,0.940395,0.009624,0.049980,Positive,1
1,2,B00813GRG4,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Negative,-1,0.000,0.138,0.862,...,Negative,-1,Negative,0.999525,-1.0,0.038600,0.508986,0.452414,Negative,-1
2,3,B000LQOCH0,4,"""Delight"" says it all",This is a confection that has been around a fe...,Positive,1,0.155,0.091,0.754,...,Positive,1,Positive,0.999765,1.0,0.898704,0.003229,0.098067,Positive,1
3,4,B000UA0QIQ,2,Cough Medicine,If you are looking for the secret ingredient i...,Negative,-1,0.000,0.000,1.000,...,Positive,1,Positive,0.999153,1.0,0.907486,0.002295,0.090219,Positive,1
4,5,B006K2ZZ7K,5,Great taffy,Great taffy at a great price. There was a wid...,Positive,1,0.448,0.000,0.552,...,Positive,1,Positive,0.998708,1.0,0.988063,0.001635,0.010302,Positive,1


In [54]:
print(f'Roberta Sentiment Analysis gives:\nAccuracy Score: {accuracy_score(df.SentimentScore, df.ScoreRoberta)}')

Roberta Sentiment Analysis gives:
Accuracy Score: 0.825


In [55]:
print(confusion_matrix(df.SentimentScore, df.ScoreRoberta))

[[ 13   3   3]
 [  7   3   8]
 [  4  10 149]]


In [56]:
print(classification_report(df.SentimentScore, df.ScoreRoberta))

              precision    recall  f1-score   support

          -1       0.54      0.68      0.60        19
           0       0.19      0.17      0.18        18
           1       0.93      0.91      0.92       163

    accuracy                           0.82       200
   macro avg       0.55      0.59      0.57       200
weighted avg       0.83      0.82      0.83       200

